## <span style="color:red; font-weight:bold">Open op Google Colab</span>


## Fine-tuning van een LLM

Het fine-tunen of trainen van een vooraf getraind model op eigen data is een proces waarbij je een bestaand model, dat op grote schaal is getraind op algemene taalkennis, aanpast aan specifieke taken of toepassingen.

**Gekwantiseerd**: Dit betekent eigenlijk dat we informatie eenvoudiger maken. Vergelijk het met een kleurenfoto omzetten in zwart-wit. Gekwantiseerd betekent dat we minder gedetailleerde informatie gebruiken, maar nog steeds de belangrijkste dingen behouden.

 Hier zijn de stappen en het doel van dit proces:

**Vooraf getraind model:**Je begint met een vooraf getraind taalmodel, zoals BERT, GPT, of een ander model dat is getraind op enorme hoeveelheden tekst uit het internet. Dit model heeft een goed begrip van taal, maar is nog niet gespecialiseerd in een specifieke taak.

**Eigen data:** Je hebt een dataset met eigen geannoteerde gegevens die specifiek zijn voor de taak of toepassing waaraan je wilt werken. Deze dataset bevat instructies en bijbehorende voorbeelden.

**Fine-tuning:** Je past het vooraf getrainde model aan door het te trainen op jouw eigen dataset. Tijdens dit proces past het model zijn interne gewichten aan om beter te presteren op de taak die in de dataset wordt behandeld. Het model leert van de eigen data en past zich aan om de specifieke taalpatronen en informatie in die gegevens beter te begrijpen.

**Specialisatie:** Na het fine-tunen is het model gespecialiseerd in de specifieke taak of toepassing waarvoor het is getraind. Het kan nu betere voorspellingen doen, tekst genereren of andere taakgerichte taken uitvoeren op basis van de eigen data.

**Evaluatie en iteratie:** Na het fine-tunen evalueer je het model om te controleren hoe goed het presteert op nieuwe, ongeziene gegevens. Je kunt het model iteratief fine-tunen en verbeteren als dat nodig is.

## Installatie

**transformers**: Dit is het Hugging Face Transformers-pakket, dat wordt gebruikt voor natuurlijke taalverwerkingstaken.

**accelerate**: Dit pakket biedt hulpmiddelen voor het versnellen van machinaal leren workflows.

**git+https://github.com/huggingface/peft.git**: Hiermee installeer je het peft-pakket van de Hugging Face GitHub-repository via Git.

datasets: Dit pakket biedt toegang tot verschillende datasets voor machinaal leren.

**bitsandbytes**: Dit is een pakket dat vermoedelijk wordt gebruikt voor Bits and Bytes-quantization, zoals eerder in de code geconfigureerd.

**einops**: Dit pakket is handig voor het uitvoeren van bewerkingen op gegevensstructuren.

**wandb**: Dit staat voor "Weights and Biases" en is een hulpmiddel voor experimenttracking en visualisatie in machinaal leren.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.

## Dataset

Om te testen heb ik op Huggingface gekozen voor een dataset vol met programmeertaken.  


In [ ]:
from datasets import load_dataset

dataset_name = "openai_humaneval"
dataset = load_dataset(dataset_name, split="test")

## Het model

Dit document bevat een korte uitleg over het gebruik van Hugging Face's Zephyr-7B-beta model met Bits and Bytes-quantization in een Google Colab-notebook. Zorg ervoor dat je de transformers-bibliotheek hebt geïnstalleerd voordat je begint.

We maken een configuratie voor Bits and Bytes-quantization met de volgende instellingen:



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

**AutoTokenizer.from_pretrained(model_name)**: Dit roept de from_pretrained-methode aan van AutoTokenizer, en het model_name is de naam van het model waarmee je de tokenizer wilt initialiseren. Het betekent dat de tokenizer wordt geconfigureerd om te werken met de specifieke tokenisatie-instructies van dat model.

**trust_remote_code=True:** Deze parameter geeft aan dat je vertrouwt op de externe broncode van het tokenizer-model. In de context van Hugging Face betekent dit dat de code voor tokenisatie op afstand wordt gebruikt in plaats van de lokale code, wat handig kan zijn voor efficiëntie.

Het **pad_token** is het speciale token dat wordt gebruikt om padding (vulling) toe te voegen aan teksten om ze dezelfde lengte te geven. Dit is handig bij het verwerken van batches met teksten van verschillende lengtes in modellen voor natuurlijke taalverwerking.

**eos_token**: Het eos_token (end-of-sequence token) is het speciale token dat aangeeft waar de tekst eindigt. Het wordt gebruikt om de scheiding tussen teksten aan te geven.

Door het pad_token in te stellen op hetzelfde teken als het eos_token, geef je aan dat de tokenizer hetzelfde token zal gebruiken om zowel het einde van de tekst als padding aan te geven. Dit kan handig zijn in sommige situaties, afhankelijk van hoe je je gegevens en modellen gebruikt.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Hieronder zullen we het configuratiebestand laden om het LoRA-model te maken. Volgens het QLoRA-artikel is het belangrijk om alle lineaire lagen in het transformer-blok te overwegen voor maximale prestaties. Daarom zullen we 'dense', 'dense_h_to_4_h' en 'dense_4h_to_h' lagen toevoegen aan de doelmodules, naast de gemengde query key value-laag."

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## De Trainer

In dit deel van de code gebruiken we de SFTTrainer uit de TRL-bibliotheek, die als een omhulsel dient voor de transformers Trainer om modellen eenvoudig af te stemmen op op instructies gebaseerde datasets met behulp van PEFT-adapters. We beginnen met het laden van de trainingsargumenten, waarmee we verschillende trainingsinstellingen specificeren, zoals de uitvoermap, batchgrootte, optimalisatiemethode en meer. Deze instellingen bepalen hoe het model wordt getraind.

PEFT staat voor "Plug and Fine-Tune", en het zijn speciale adapters die in staat zijn om pre-getrainde taalmodellen aan te passen voor specifieke taakgerichte of domeinspecifieke taken. Deze adapters kunnen worden "ingeplugd" in een bestaand pre-getraind model zonder dat het nodig is om het hele model opnieuw te trainen.

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)


**train_dataset**: Dit is het trainingsdataset dat wordt gebruikt om het model te fine-tunen. Het moet bestaan uit instructies en bijbehorende gegevens voor de taak.

**peft_config**: Dit is de configuratie van de PEFT-adapters die je mogelijk hebt gedefinieerd om het model aan te passen aan de taak. Het kan parameters bevatten die de werking van de adapters beïnvloeden.

**dataset_text_field**: Dit geeft aan welk veld in de trainingsdataset de tekstinformatie bevat.

**max_seq_length**: Dit is de maximale lengte van de sequenties, zoals hierboven gedefinieerd.

**tokenizer**: De tokenizer wordt gebruikt om tekst in stukken te verdelen en te vertalen naar geschikte invoer voor het model.

**args=training_arguments**: Dit zijn de trainingsargumenten die zijn geconfigureerd, zoals batchgrootte, optimalisatiemethode, enz., zoals eerder gedefinieerd in de code. Ze bepalen hoe het model wordt getraind.

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="test",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.038000
20,0.759500
30,0.664800
40,0.591800
50,0.535100
60,0.488200
70,0.436600
80,0.366800
90,0.345000
100,0.277600


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss
10,1.038000
20,0.759500
30,0.664800
40,0.591800
50,0.535100
60,0.488200
70,0.436600
80,0.366800
90,0.345000
100,0.277600


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=500, training_loss=0.17714973038434983, metrics={'train_runtime': 8886.9366, 'train_samples_per_second': 0.9, 'train_steps_per_second': 0.056, 'total_flos': 7.79730156403753e+16, 'train_loss': 0.17714973038434983, 'epoch': 48.78})

Tijdens de training zou het model soepel moeten convergeren, zoals te zien is in onderstaande afbeelding:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

De SFTTrainer zorgt er ook voor dat tijdens de training alleen de adapters op de juiste manier worden opgeslagen, in plaats van het volledige model op te slaan. Dit zorgt voor efficiëntie en ruimtebesparing bij het opslaan van de getrainde modellen.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
# model = get_peft_model(model, lora_config)